In [ ]:
#멀티스레드,타임슬립(랜덤 1~3초)
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime
import time
import threading
import random
from concurrent.futures import ThreadPoolExecutor

sc_sdate = '2014-09-01'  
sc_edate = '2014-09-02'  

# 사이트(기간 입력 후)
url = f"https://news.einfomax.co.kr/news/articleList.html?page=1&total=6417&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=A&sc_level=&sc_article_type=&sc_view_level=&sc_sdate={sc_sdate}&sc_edate={sc_edate}&sc_serial_number=&sc_word=%EA%B8%88%EB%A6%AC&box_idxno=&sc_multi_code=&sc_is_image=&sc_is_movie=&sc_user_name=&sc_order_by=E&view_type=sm"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Referer': 'https://www.nate.com/'
}
base = 'https://news.einfomax.co.kr'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
total = soup.select_one('#sections > section > header > h3 > small').text
total_num = ''.join([i for i in total if i.isdigit()])
total_num = int(total_num)
pages = (total_num + 19) // 20  # 전체 페이지를 계산

#기사 저장파일 만들기
output_dir = r"C:\Users\cobe2\Documents\SeSAC\PJ\info"
os.makedirs(output_dir, exist_ok=True)

# 한 스레드가 파일 저장시 다른 스레드가 저장 못함
file_lock = threading.Lock()

def crawl_page(page):
    try:
        url = f"https://news.einfomax.co.kr/news/articleList.html?page={page}&total=6417&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=A&sc_level=&sc_article_type=&sc_view_level=&sc_sdate={sc_sdate}&sc_edate={sc_edate}&sc_serial_number=&sc_word=%EA%B8%88%EB%A6%AC&box_idxno=&sc_multi_code=&sc_is_image=&sc_is_movie=&sc_user_name=&sc_order_by=E&view_type=sm"
        response = requests.get(url, headers=headers)
        time.sleep(random.uniform(1, 3))
        soup = BeautifulSoup(response.text, 'html.parser')
        li_tg = soup.select('ul.type2>li>h4.titles>a')  # 뉴스기사 링크 리스트로
        for i in li_tg:
            target = i.attrs['href']#dictionary의 key로 value를 호출
            crawling_url = base + target
            response = requests.get(crawling_url, headers=headers)
            crawling_soup = BeautifulSoup(response.text, 'html.parser')  # 해당 뉴스기사 링크의 html 정보 추출
            title = crawling_soup.select_one('h3.heading').text
            new_title = title.replace("/", "_") 

            date_li = crawling_soup.select('ul.infomation>li')[1].text 
            date = date_li.split("입력")[-1].replace('.', '_').replace(':', '_')

            info = crawling_soup.select_one('#article-view-content-div').text.replace('\n', '').replace('\r', '').replace('\t', '')

            file_path = os.path.join(output_dir, f"{date}_연합인포맥스.text")
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(f'제목\n{title}\n\n내용\n{info}\n웹싸이트\n{crawling_url}\n')
                print(f'Saved file: {file_path}')
                time.sleep(2)
    
    except Exception as e:
        print(f"Error occurred on page {page}: {e}")
    
# 멀티스레드
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(crawl_page, range(1, pages + 1))

print("Crawling End")